#### World Population Prospects 2019
#### File POP/1-1: Total population (both sexes combined) by region, subregion and country, ####annually for 1950-2100 (thousands)
#### Constant-mortality variant, 2020 - 2100
#### POP/DB/WPP/Rev.2019/POP/F01-1
#### © August 2019 by United Nations, made available under a Creative Commons license CC BY 3.0 IGO: #### http://creativecommons.org/licenses/by/3.0/igo/
#### United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects 2019, Online Edition. Rev. 1.

In [1]:
import pandas as pd
import requests
import os
import csv
from googletrans import Translator

In [2]:
pd.set_option("display.precision", 2)

### OMS https://covid19.who.int/WHO-COVID-19-global-data.csv

In [3]:
direccion_url = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'

'''Uncomment para definir el Path'''

direccion_local = '/Users/Dr.LuisEvelioRestrepoGarcia/datosRecopilador/CovidMundo.csv'    # En mi Apple
'''Obtiene la base de datos actualizada y la guardar en disco local.'''

datos_req = requests.get(direccion_url)
try:
    datos_req.raise_for_status()
except Exception as exc:
    print('Problema en la descarga: %s' % (exc))
print(datos_req.status_code == requests.codes.ok)
playFile = open('CovidMundo.csv', 'wb')
for chunk in datos_req.iter_content(100000):
    playFile.write(chunk)
playFile.close()

covMun = pd.read_csv('CovidMundo.csv', usecols=['Date_reported', ' Country', 
       ' New_cases', ' Cumulative_cases', ' New_deaths', ' Cumulative_deaths'])

covMun.rename({'Date_reported': 'Fecha_reporte', ' Country': 'País', ' New_cases':'Total_Casos', 
                ' Cumulative_cases': 'Casos_acumulados', ' New_deaths':'Fallecidos', 
                ' Cumulative_deaths':'Muertes_acumuladas'}, axis='columns', inplace=True)
covMun = covMun.groupby('País')[['Total_Casos','Fallecidos']].sum()


covMun = covMun.reset_index()

covMun

True


,País,Total_Casos,Fallecidos
0,Afghanistan,38883,1437
1,Albania,11948,347
2,Algeria,49194,1654
3,Andorra,1483,53
4,Angola,3789,144
...,...,...,...
211,Viet Nam,1066,35
212,Yemen,2026,586
213,Zambia,13928,326
214,Zimbabwe,7633,224


In [4]:
translator = Translator()
covMun['Spanish'] = covMun['País'].apply(translator.translate, src='en', dest='es').apply(getattr, args=('text',))
covMun.to_csv('CovidMundoT.csv')
covMun.drop('País', axis= 'columns', inplace=True)
covMun.rename({'Spanish':'País'}, axis = 'columns', inplace=True)
covMun.set_index('País', inplace=True)

In [5]:
mundo= pd.read_csv('poblacionMundial.csv')
mundo.rename({'Spanish':'País'}, axis='columns', inplace=True)
mundo.set_index('País', inplace=True)
mundo = mundo['2020']

In [6]:
Tasas_mundiales = pd.merge(covMun, mundo, left_index=True, right_index=True)

Tasas_mundiales.rename({'2020':'Población'}, axis ='columns', inplace=True)

Tasas_mundiales['Tasa de mortalidad por millón de habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Población'])*1000000
Tasas_mundiales['Tasa de letalidad por cien habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Total_Casos'])*100

In [7]:
Tasas_mundiales

,Total_Casos,Fallecidos,Población,Tasa de mortalidad por millón de habitantes,Tasa de letalidad por cien habitantes
País,,,,,
Afganistán,38883,1437,38928000,36.91,3.70
Albania,11948,347,2878000,120.57,2.90
Argelia,49194,1654,43851000,37.72,3.36
Andorra,1483,53,77000,688.31,3.57
Angola,3789,144,32866000,4.38,3.80
...,...,...,...,...,...
Venezuela (República Bolivariana de),63416,511,28436000,17.97,0.81
Vietnam,1066,35,97339000,0.36,3.28
Yemen,2026,586,29826000,19.65,28.92


In [8]:
pais = ['Colombia', 'Brasil', 'Argentina', 'Chile',
        'Estados Unidos de America', 'Canadá',
        'España', 'Francia', 'Italia', 
        'Alemania', 'Suecia']

In [9]:
Tasas_mundiales.loc[pais]

,Total_Casos,Fallecidos,Población,Tasa de mortalidad por millón de habitantes,Tasa de letalidad por cien habitantes
País,,,,,
Colombia,736377,23478,50883000,461.41,3.19
Brasil,4419083,134106,212559000,630.91,3.03
Argentina,589012,12229,45196000,270.58,2.08
Chile,441150,12142,19116000,635.17,2.75
Estados Unidos de America,6571119,195638,331003000,591.05,2.98
Canadá,139747,9193,37742000,243.57,6.58
España,625651,30405,46755000,650.30,4.86
Francia,394306,30930,65274000,473.85,7.84
Italia,293025,35658,60462000,589.76,12.17
